# Dependencies

In [1]:
# Dependencies 
from gazpacho import Soup
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Web Scrape csv information

In [2]:
# Run Chrome to assist with identifying another set of html calls.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/normanadkins/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [3]:
# Visit the new URL for preview
url = "https://s3.amazonaws.com/tripdata/index.html"
browser.visit(url)

In [4]:
# Requests
html_page = requests.get(url)
soup = BeautifulSoup(html_page.content, "html.parser")

In [5]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Create an list of csv links
rows = []

# Loop through the links to append 
for link in soup.find_all('a'):
    
    # Append the zip "clean" links
    rows.append(link.get('href'))

# Only capture the zip files that are the focus.
links = rows[92:98]

# Print to confirm the changes
links

['https://s3.amazonaws.com/tripdata/202101-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202102-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202103-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202104-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202105-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202106-citibike-tripdata.csv.zip']

In [6]:
# Close out browser session
browser.quit()

# Unzip and load csv data into a single DataFrame.

## Note:  This takes some computing power to execute.

In [7]:
# Start a DataFrame
master_df = pd.DataFrame()

# Only do one at a time due to memory limitation
for link in links:
    # open url
    resp = urlopen(link)
    # read zip
    zipfile = ZipFile(BytesIO(resp.read()))
    # Get csv file name
    fname = zipfile.namelist()[0]
    # Data Frame
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    # Close zip
    zipfile.close()
    # Concatenate DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True, sort=False)
    # Print Confirmation
    print(f'{fname} has been completed')
    
# Save Data Frame for Tableau Visuals
master_df.to_csv("bike_rental.csv")

# Print Save Confirmation
print("document has been saved")

202101-citibike-tripdata.csv has been completed
202102-citibike-tripdata.csv has been completed
202103-citibike-tripdata.csv has been completed
202104-citibike-tripdata.csv has been completed
202105-citibike-tripdata.csv has been completed
202106-citibike-tripdata.csv has been completed
document has been saved


In [8]:
# Free up Memory by deleting a DataFrame no longer needed
del df

# Data Cleansing

## Assess the data

In [9]:
# Preview Data
master_df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,2513,2021-01-01 00:00:11.9020,2021-01-01 00:42:05.2260,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2519,2021-01-01 00:00:15.0960,2021-01-01 00:42:14.9780,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1207,2021-01-01 00:00:28.9300,2021-01-01 00:20:36.6510,3144,E 81 St & Park Ave,40.77677702,-73.9590097,3724,7 Ave & Central Park South,40.7667405590595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2506,2021-01-01 00:00:32.7130,2021-01-01 00:42:19.3980,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,959,2021-01-01 00:00:35.3650,2021-01-01 00:16:34.6010,534,Water - Whitehall Plaza,40.70255065,-74.0127234,332,Cherry St,40.71219906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Length of DataFrame for considering how many rows would be lost for specific columns.
master_df.shape[0]

11245774

In [11]:
# Count Null Values by column
master_df.isnull().sum(axis=0)

tripduration               10150428
starttime                  10150428
stoptime                   10150428
start station id           10150428
start station name         10150428
start station latitude     10150428
start station longitude    10150428
end station id             10150428
end station name           10150428
end station latitude       10150428
end station longitude      10150428
bikeid                     10150428
usertype                   10150428
birth year                 10150428
gender                     10150428
ride_id                     1095346
rideable_type               1095346
started_at                  1095346
ended_at                    1095346
start_station_name          1095714
start_station_id            1095714
end_station_name            1119157
end_station_id              1119157
start_lat                   1095346
start_lng                   1095346
end_lat                     1116900
end_lng                     1116900
member_casual               

## Cleaning the Data such as dropping duplicate columns, removing null values, and changing the datatypes.

In [12]:
# Drop Duplicate Columns that are primarily empty anyway.
master_df = master_df.drop(master_df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]], axis=1)

In [13]:
# Confirm that columns were dropped.
master_df.isnull().sum(axis=0)

ride_id               1095346
rideable_type         1095346
started_at            1095346
ended_at              1095346
start_station_name    1095714
start_station_id      1095714
end_station_name      1119157
end_station_id        1119157
start_lat             1095346
start_lng             1095346
end_lat               1116900
end_lng               1116900
member_casual         1095346
dtype: int64

In [15]:
# After seeing updated preview, dropping null values in other columns
master_df = master_df.dropna(subset=["ride_id", "rideable_type", "member_casual", "start_lat", "start_lng", 
                                     "end_lat", "end_lng", "started_at", "ended_at", "start_station_name", 
                                     "start_station_id", "end_station_name", "end_station_id"])

In [16]:
# Confirm that nulls were dropped.
master_df.isnull().sum(axis=0)

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [17]:
# Now that we have a clean data set, the data types need to be reviewed for accuracy.
master_df.dtypes

ride_id               object
rideable_type         object
started_at            object
ended_at              object
start_station_name    object
start_station_id      object
end_station_name      object
end_station_id        object
start_lat             object
start_lng             object
end_lat               object
end_lng               object
member_casual         object
dtype: object

In [18]:
# New Dependency
import datetime

# Convert datatype for started_at to datetime
master_df["started_at"].astype('datetime64[s]')

1095346    2021-02-26 16:38:54
1095347    2021-02-17 11:09:11
1095348    2021-02-26 18:33:29
1095349    2021-02-26 12:48:35
1095350    2021-02-25 17:23:22
                   ...        
11245769   2021-06-30 13:13:05
11245770   2021-06-23 22:51:46
11245771   2021-06-19 16:18:58
11245772   2021-06-23 11:29:37
11245773   2021-06-19 00:54:21
Name: started_at, Length: 10126607, dtype: datetime64[ns]

In [19]:
# Convert datatype for ended_at to datetime
master_df["ended_at"].astype('datetime64[s]')

1095346    2021-02-26 16:44:37
1095347    2021-02-17 11:26:47
1095348    2021-02-26 19:05:41
1095349    2021-02-26 13:07:24
1095350    2021-02-25 17:28:20
                   ...        
11245769   2021-06-30 13:25:54
11245770   2021-06-23 22:56:56
11245771   2021-06-19 16:33:32
11245772   2021-06-23 11:30:59
11245773   2021-06-19 01:02:39
Name: ended_at, Length: 10126607, dtype: datetime64[ns]

# Export the Clean csv to use for Tableau Visualizations.

In [20]:
# Save Data Frame for Tableau Visuals
master_df.to_csv("bike_rental.csv")